In [9]:
import sys
import subprocess

exit_code = subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", "requirements.txt"])
if exit_code:
    raise Exception("Non-zero return code when installing the dependencies, please try reinstalling them manually")

Defaulting to user installation because normal site-packages is not writeable


In [10]:
import boto3
import requests
ec2_client = boto3.client("ec2", "us-east-1")
group_name = "hillel-lesson20-2-security-group"

my_cidr = f'{requests.get("https://api.ipify.org?format=json").json().get("ip", "127.0.0.1")}/32'
print(my_cidr)
output = ec2_client.create_security_group(
    Description="Security Group for Lesson 20",
    GroupName=group_name,
    VpcId="vpc-0565475cef4feac4e"
)

group_id = output.get('GroupId')
print("Successfully created security group")

ec2_client.authorize_security_group_ingress(
    GroupId=group_id,
    IpPermissions=[{
        "IpProtocol": "tcp",
        "FromPort": 80,
        "ToPort": 80,
        "IpRanges": [{
            "CidrIp": my_cidr,
            "Description": "HTTP from my laptop"
        }]
    }, {
        "IpProtocol": "tcp",
        "FromPort": 22,
        "ToPort": 22,
        "IpRanges": [{
            "CidrIp": my_cidr,
            "Description": "SSH from my laptop"
        }]
    },
        {
        "IpProtocol": "tcp",
        "FromPort": 8081,
        "ToPort": 8081,
        "IpRanges": [{
            "CidrIp": my_cidr,
            "Description": "HTTP from my laptop"
        }]
    }
    ]
)

195.64.239.131/32
Successfully created security group


{'Return': True,
 'SecurityGroupRules': [{'SecurityGroupRuleId': 'sgr-013ca7e9a797a277a',
   'GroupId': 'sg-02ee0b3cc61d4c15b',
   'GroupOwnerId': '633143608043',
   'IsEgress': False,
   'IpProtocol': 'tcp',
   'FromPort': 80,
   'ToPort': 80,
   'CidrIpv4': '195.64.239.131/32',
   'Description': 'HTTP from my laptop'},
  {'SecurityGroupRuleId': 'sgr-03170fd9e4cbbbcb8',
   'GroupId': 'sg-02ee0b3cc61d4c15b',
   'GroupOwnerId': '633143608043',
   'IsEgress': False,
   'IpProtocol': 'tcp',
   'FromPort': 22,
   'ToPort': 22,
   'CidrIpv4': '195.64.239.131/32',
   'Description': 'SSH from my laptop'},
  {'SecurityGroupRuleId': 'sgr-06099232297163fe9',
   'GroupId': 'sg-02ee0b3cc61d4c15b',
   'GroupOwnerId': '633143608043',
   'IsEgress': False,
   'IpProtocol': 'tcp',
   'FromPort': 8081,
   'ToPort': 8081,
   'CidrIpv4': '195.64.239.131/32',
   'Description': 'HTTP from my laptop'}],
 'ResponseMetadata': {'RequestId': '17fd149a-57bc-4767-b005-01233e3f0567',
  'HTTPStatusCode': 200,
  'HT

In [11]:
import boto3
ami_id = "ami-03f7420f29bc510fd"
output = ec2_client.run_instances(
  MaxCount=1,
  MinCount=1,
  InstanceType="t2.micro",
  ImageId=ami_id,
  TagSpecifications=[
        {
            "ResourceType": 'instance',
            "Tags": [
                {
                    'Key': 'klimadron-hw15',
                    'Value': 'HW15-AndriiKlimakhin-Instance'
                },
            ]
        }
  ],
#    KeyName="Name",
  IamInstanceProfile={
      "Name": "hillel-les20-role"
   },
  SecurityGroupIds=[
      group_id,
   ],
#   UserData="""
# #!/usr/bin/env bash
# docker run -d --name nginx -p 80:80 nginx:alpine
#   """ 
)
instance_ids = list(map(lambda i : i.get("InstanceId"), output.get("Instances")))
print(f"Launched instances with ids: {' '.join(instance_ids)}")
jdun = ec2_client.get_waiter("instance_running")
jdun.wait(InstanceIds=instance_ids)
print(f"Instances with ids: {' '.join(instance_ids)} are up and running")

Launched instances with ids: i-0dcb97ae530abdee6
Instances with ids: i-0dcb97ae530abdee6 are up and running


In [12]:
ec2_resource = boto3.resource("ec2", "us-east-1")
instance = ec2_resource.Instance(instance_ids[0])
print(instance.public_ip_address)

34.204.3.239


**Terminate** instances and wait until they are down:

In [13]:
# ec2_client.terminate_instances(InstanceIds=instance_ids)
# jdun = ec2_client.get_waiter("instance_terminated")
# jdun.wait(InstanceIds=instance_ids)
# print(f"Instances with ids: {' '.join(instance_ids)} are terminated")